In [1]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle as cPickle  # Sorry, workaround for Python 3.4
import gzip

# Third-party libraries
import numpy as np

def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding='latin1')  # Make it work with Py3.x
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [2]:
%time training_data, validation_data, test_data = load_data_wrapper()

CPU times: user 1.65 s, sys: 487 ms, total: 2.13 s
Wall time: 2.15 s


In [3]:
%time x, y = zip(*training_data)

x = np.array(x)
x = x.reshape(50000, 784)

y = np.array(y)
y = y.reshape(50000, 10)

CPU times: user 56.6 ms, sys: 24.7 ms, total: 81.3 ms
Wall time: 82.5 ms


In [4]:
import keras


import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils

from sklearn.preprocessing import StandardScaler

np.random.seed(1337) # for reproducibility


def preprocess_data(X, scaler=None):
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler



Using Theano backend.


/Users/tjl/Development/sandpit/lca16/source_packages/Theano/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [5]:
 
nb_classes = y.shape[1]
print(nb_classes, 'classes')

dims = x.shape[1]
print(dims, 'dims')

 


10 classes
784 dims


In [8]:

print("Building model...")

LAYER_ONE_SIZE = 1024

model = Sequential()
model.add(Dense(output_dim=LAYER_ONE_SIZE, input_dim=dims, init='glorot_uniform'))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.5))

# model.add(Dense(LAYER_ONE_SIZE, LAYER_ONE_SIZE, init='glorot_uniform'))
# model.add(PReLU((LAYER_ONE_SIZE,)))
# model.add(BatchNormalization((LAYER_ONE_SIZE,)))
# model.add(Dropout(0.5))

# model.add(Dense(LAYER_ONE_SIZE, LAYER_ONE_SIZE, init='glorot_uniform'))
# model.add(PReLU((LAYER_ONE_SIZE,)))
# model.add(BatchNormalization((LAYER_ONE_SIZE,)))
# model.add(Dropout(0.5))

model.add(Dense(output_dim=nb_classes, input_dim=LAYER_ONE_SIZE, init='glorot_uniform'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam")

print("Training model...")

%time model.fit(x, y, nb_epoch=8, batch_size=128, validation_split=0.15)  

Building model...
Training model...
Train on 42500 samples, validate on 7500 samples
Epoch 1/8
42500/42500 [==============================] - 7s - loss: 0.5229 - val_loss: 0.2689
Epoch 2/8
42500/42500 [==============================] - 9s - loss: 0.2152 - val_loss: 0.1900
Epoch 3/8
42500/42500 [==============================] - 9s - loss: 0.1485 - val_loss: 0.1473
Epoch 4/8
42500/42500 [==============================] - 9s - loss: 0.1120 - val_loss: 0.1275
Epoch 5/8
42500/42500 [==============================] - 9s - loss: 0.0910 - val_loss: 0.1130
Epoch 6/8
42500/42500 [==============================] - 9s - loss: 0.0746 - val_loss: 0.1097
Epoch 7/8
42500/42500 [==============================] - 8s - loss: 0.0649 - val_loss: 0.1095
Epoch 8/8
42500/42500 [==============================] - 7s - loss: 0.0579 - val_loss: 0.1077
CPU times: user 1min 40s, sys: 3.43 s, total: 1min 44s
Wall time: 1min 10s


In [9]:
p_x, p_y = zip(*test_data)

p_x = np.array(p_x)
p_x = p_x.reshape(10000, 784)

p_y = np.array(p_y)
p_y = p_y.reshape(10000)

In [17]:
%time preds = model.predict(p_x)
print(preds[1])
print(np.argmax(preds[1]))

CPU times: user 547 ms, sys: 13.2 ms, total: 561 ms
Wall time: 240 ms
[  4.02844989e-06   2.72390957e-04   9.99713719e-01   5.38024096e-06
   4.17215018e-09   2.35322858e-07   1.78867117e-06   4.43746873e-10
   2.44387229e-06   1.14954324e-10]
2


In [18]:
pred_idx = [np.argmax(a) for a in preds]

In [19]:
pairs = zip(pred_idx, p_y)
print(len(p_y))

10000


In [20]:
number_correct = sum([int(a == b) for a, b in pairs])
print(number_correct)

9764


In [21]:
number_incorrect = len(p_y) - number_correct
print(number_incorrect)

236


In [22]:
eps = 0.000001  # avoid division by zero
success_rate = number_correct / float((number_correct + number_incorrect + eps))
print(success_rate)

0.9763999999023599
